<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j4 

In [2]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

In [3]:
#@title Baixando base de dados proteína ligante {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/pyLiBELa/SB/'

with open(path+'list.txt') as f:
    pdb_codes_list = f.readlines()

pdb_codes_list = [pdb_code[0:4] for pdb_code in pdb_codes_list]

In [4]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 5.0 #@param {type:"number"}
Input.y_dim = 5.0 #@param {type:"number"}
Input.z_dim = 5.0 #@param {type:"number"}
Input.write_grids = True #@param {type:"boolean"}
Input.dock_parallel = True #@param {type:"boolean"}
Input.parallel_jobs = 4 #@param {type:"integer"}
Input.write_mol2 = True #@param {type:"boolean"};

In [8]:
import os
import timeit
import numpy as np

num_pairs = 2
num_metrics = 4

spacing_grid_list = [0.1,0.3]
length_input=len(spacing_grid_list)


metrics_matrix=np.zeros((length_input*num_pairs,num_metrics+1))

dim_grid = 3

i=0
for j in range(num_pairs):
  for spacing_grid in spacing_grid_list:
    pdb_code = pdb_codes_list[j]

    print(pdb_code,spacing_grid)

    Input.x_dim, Input.y_dim, Input.z_dim = dim_grid, dim_grid, dim_grid


    metrics_list = np.zeros(num_metrics+1)

    metrics_list[0] = j
    
    metrics_list[1] = spacing_grid

    Input = PARSER()
    Input.grid_prefix = "McGrid_"+pdb_code

    Writer = WRITER(Input)
    Coord = COORD_MC()
    HB = FindHB()

    lig_src =  path + pdb_code +'/' + pdb_code + '.lig.am1bcc.mol2.gz'
    rec_src = path + pdb_code +'/' + pdb_code + '.rec.clean.mol2.gz'

    lig = Mol2(Input, lig_src)
    rec = Mol2(Input, rec_src)
    ref_lig = Mol2(Input, lig_src)

    Energy = Energy2(Input)

    for k in range(len(list(rec.residue_pointer))-1):
      HB.parse_residue(rec.residue_pointer[k]-1, rec.residue_pointer[k+1]-2, rec.resnames[k], rec, lig, 9.0)

    HB.find_ligandHB(lig_src, lig)

    com = Coord.compute_com(lig)

    energy1 = Energy.compute_ene(rec, lig, lig.xyz)
    print("Original energy: %7.3f" % energy1)
    metrics_list[2] = energy1

    Input.write_grids = True

    start_time = timeit.default_timer()
    Grids = Grid(Input, Writer, rec, com)
    time = timeit.default_timer() - start_time
    print('Timeit: %.2f' %time)
    metrics_list[3] = time

    Writer.write_box(com, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)

    energy2 = Energy.compute_ene(Grids, lig, lig.xyz)
    print("Grid energy: %7.3f" % energy2)
    metrics_list[4] = energy2

    folder = path + pdb_code
    !mv *.grid McLiBELa.log box.pdb $folder

    #print(metrics_list)
    print('')

    metrics_matrix[i] = metrics_list
    i+=1


121P 0.1
Original energy: -237.910
Timeit: 28.08
Grid energy: 2999795.409

181L 0.1
Original energy: -12.322
Timeit: 24.40
Grid energy: -12.179

121P 0.3
Original energy: -237.910
Timeit: 25.77
Grid energy: 2999795.409

181L 0.3
Original energy: -12.322
Timeit: 26.35
Grid energy: -12.179



In [32]:
print("{:<10} {:<15} {:<17} {:<10} {:<10}".format('PDB_code','Grid spacing','Original Energy','Time','Grid Energy'))
for i in range(num_pairs+length_input):
  metrics_line = metrics_matrix[i]
  pdb_code_print = pdb_codes_list[int(metrics_line[0])]

  print("{:<10} {:<15} {:<17} {:<10} {:<10}".format(pdb_code_print,metrics_line[1],'%7.3f'%metrics_line[2],'%.2f'%metrics_line[3], '%7.3f'%metrics_line[4])) 



PDB_code   Grid spacing    Original Energy   Time       Grid Energy
121P       0.1             -237.910          28.08      2999795.409
181L       0.1             -12.322           24.40      -12.179   
121P       0.3             -237.910          25.77      2999795.409
181L       0.3             -12.322           26.35      -12.179   


In [42]:
#Writing data

data_path = '/content/drive/MyDrive/pyLiBELa/'
data_name = 'grid_spacing_test.dat'

data = open(data_name,'w')

variables='#'
#variables+='\t'.join(metrics_name_list)
variables+="{:<10} {:<15} {:<17} {:<10} {:<10}".format('PDB_code','Grid spacing','Original Energy','Time','Grid Energy')
variables+='\n'
data.write(variables)
data_line=''


for i in range(num_pairs+length_input):
  metrics_line = metrics_matrix[i]
  pdb_code_print = pdb_codes_list[int(metrics_line[0])]

  data_line+="{:<10} {:<15} {:<17} {:<10} {:<10}".format(pdb_code_print,metrics_line[1],'%7.3f'%metrics_line[2],'%.2f'%metrics_line[3], '%7.3f'%metrics_line[4]) 

  data_line+='\n'

  #data_line+="{}\t{}\t{}\t{}\t{}".format(pdb_code_print,metrics_line[1],'%7.3f'%metrics_line[2],'%.2f'%metrics_line[3], '%7.3f'%metrics_line[4]) 
  #data_line+= '\t'

data.write(data_line)
data.close()

!mv $data_name $data_path

In [41]:
data_loc = data_path + data_name
!more $data_loc

#PDB_code   Grid spacing    Original Energy   Time       Grid Energy
121P       0.1             -237.910          28.08      2999795.409
181L       0.1             -12.322           24.40      -12.179   
121P       0.3             -237.910          25.77      2999795.409
181L       0.3             -12.322           26.35      -12.179   


In [43]:
#Writing data in csv
import csv

data_path = '/content/drive/MyDrive/pyLiBELa/'
data_name_csv = 'grid_spacing_test.csv'


metrics_name_list = ['PDB_code','Grid spacing','Original Energy','Time','Grid Energy']
with open(data_name_csv,'w', encoding='UTF8', newline='') as data:

  writer = csv.writer(data)

  writer.writerow(metrics_name_list)

  for i in range(num_pairs+length_input):
    metrics_line = metrics_matrix[i]
    pdb_code_print = pdb_codes_list[int(metrics_line[0])]
    row = [pdb_code_print,metrics_line[1],'%7.3f'%metrics_line[2],'%.2f'%metrics_line[3], '%7.3f'%metrics_line[4]]
    writer.writerow(row)


!mv $data_name_csv $data_path